1. 630710438 พรชิตา สุขใจ
2. 630710439 พรดนัย โมชัย

In [1]:
import gensim
gensim.__version__

'4.3.2'

In [2]:
import pandas as pd
import numpy as np
import re
from numpy.linalg import norm
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


แหล่งที่มาข้อมูล : https://www.azlyrics.com/c/charlieputh.html?fbclid=IwAR0JSn4zCJIAODzVHRhDIUrZ4nJP9YpPRUXnHUzYXNx-G2BHdcV2mBDd2ew

In [3]:
df = pd.read_csv("C:/Users/User/Google Drive/NLP/charlie_puth.csv")

In [4]:
df

,Artist,Lyrics,Song
0,Charlie Puth,There's a moment when you find out Who your...,The Moment
1,Charlie Puth,Its 10 p.m. and I am missing My seventh son...,I Suck At Writing Lyrics
2,Charlie Puth,"Ohh Yeah I don't wanna hurt you baby, no m...",I Don't Wanna Hurt You Baby (Acoustic)
3,Charlie Puth,Ohh Doo doo doo doo doo Right next to you ...,Next To You
4,Charlie Puth,Oh-oh Doo doo doo doo doo Doo doo doo doo d...,Time Passes By
...,...,...,...
97,Charlie Puth,You shout it out But I can't hear a word yo...,Titanium
98,Charlie Puth,I'll miss you singing Stevie Wonder Isn't s...,Two Months
99,Charlie Puth,Red cups and sweaty bodies everywhere Hands...,We Can't Stop
100,Charlie Puth,I took her out to dinner I thought I'd had ...,When She Kissed Me


In [5]:
df = df.applymap(lambda x: x.lower() if type(x) == str else x)

In [6]:
df['english_lang'] = df['Lyrics'].apply(lambda x: all(char.isascii() for char in x))

In [7]:
df['Lyrics'] = df['Lyrics'].apply(lambda x: re.sub(r'[^a-zA-Z\s\']', '', x))

In [8]:
df['english_lang'].unique()

array([ True, False])

In [9]:
df[df['english_lang'] == False]

,Artist,Lyrics,Song,english_lang
6,charlie puth,oh darling i know you're taken something 'b...,i won't tell a soul,False
14,charlie puth,charlie puth we don't talk anymore we don't ...,we don't talk anymore,False
15,charlie puth,i'd stroll into a bank and put a ski mask o...,my gospel,False
57,charlie puth,'cause i'm all that you want girl all that ...,bon appã©tit,False
73,charlie puth,oh im facing the bottle oh for all of my pr...,instagram models,False
76,charlie puth,stole my heart and i fell for you always kn...,l.u.v.,False


In [10]:
df['tokenizer'] = df['Lyrics'].apply(word_tokenize)

In [11]:
df

,Artist,Lyrics,Song,english_lang,tokenizer
0,charlie puth,there's a moment when you find out who your...,the moment,True,"[there, 's, a, moment, when, you, find, out, w..."
1,charlie puth,its pm and i am missing my seventh song fo...,i suck at writing lyrics,True,"[its, pm, and, i, am, missing, my, seventh, so..."
2,charlie puth,ohh yeah i don't wanna hurt you baby no mo...,i don't wanna hurt you baby (acoustic),True,"[ohh, yeah, i, do, n't, wan, na, hurt, you, ba..."
3,charlie puth,ohh doo doo doo doo doo right next to you ...,next to you,True,"[ohh, doo, doo, doo, doo, doo, right, next, to..."
4,charlie puth,ohoh doo doo doo doo doo doo doo doo doo do...,time passes by,True,"[ohoh, doo, doo, doo, doo, doo, doo, doo, doo,..."
...,...,...,...,...,...
97,charlie puth,you shout it out but i can't hear a word yo...,titanium,True,"[you, shout, it, out, but, i, ca, n't, hear, a..."
98,charlie puth,i'll miss you singing stevie wonder isn't s...,two months,True,"[i, 'll, miss, you, singing, stevie, wonder, i..."
99,charlie puth,red cups and sweaty bodies everywhere hands...,we can't stop,True,"[red, cups, and, sweaty, bodies, everywhere, h..."
100,charlie puth,i took her out to dinner i thought i'd had ...,when she kissed me,True,"[i, took, her, out, to, dinner, i, thought, i,..."


In [12]:
df['length_word'] = df['tokenizer'].apply(len)

In [13]:
df

,Artist,Lyrics,Song,english_lang,tokenizer,length_word
0,charlie puth,there's a moment when you find out who your...,the moment,True,"[there, 's, a, moment, when, you, find, out, w...",190
1,charlie puth,its pm and i am missing my seventh song fo...,i suck at writing lyrics,True,"[its, pm, and, i, am, missing, my, seventh, so...",283
2,charlie puth,ohh yeah i don't wanna hurt you baby no mo...,i don't wanna hurt you baby (acoustic),True,"[ohh, yeah, i, do, n't, wan, na, hurt, you, ba...",421
3,charlie puth,ohh doo doo doo doo doo right next to you ...,next to you,True,"[ohh, doo, doo, doo, doo, doo, right, next, to...",311
4,charlie puth,ohoh doo doo doo doo doo doo doo doo doo do...,time passes by,True,"[ohoh, doo, doo, doo, doo, doo, doo, doo, doo,...",257
...,...,...,...,...,...,...
97,charlie puth,you shout it out but i can't hear a word yo...,titanium,True,"[you, shout, it, out, but, i, ca, n't, hear, a...",157
98,charlie puth,i'll miss you singing stevie wonder isn't s...,two months,True,"[i, 'll, miss, you, singing, stevie, wonder, i...",254
99,charlie puth,red cups and sweaty bodies everywhere hands...,we can't stop,True,"[red, cups, and, sweaty, bodies, everywhere, h...",408
100,charlie puth,i took her out to dinner i thought i'd had ...,when she kissed me,True,"[i, took, her, out, to, dinner, i, thought, i,...",338


In [14]:
print("Minimum Length Word",df['length_word'].min())
print("Maximunm Length Word",df['length_word'].max())
print("Average Length Word",int(df['length_word'].mean()))

Minimum Length Word 146
Maximunm Length Word 637
Average Length Word 347


# Train Word2Vec model with data web scraping

- default value of min_count=5
- vector_size is the number of dimensions (N) of the N-dimensional space that gensim Word2Vec maps the words onto.

In [15]:
modelword2vec = Word2Vec(sentences=df['tokenizer'], vector_size=int(df['length_word'].mean()), window=5, min_count=5, workers=4)

word 
- baby
- beautiful

In [16]:
vec_baby = modelword2vec.wv['baby']
vec_beautiful = modelword2vec.wv['beautiful']
cosine = np.dot(vec_baby,vec_beautiful)/(norm(vec_baby)*norm(vec_beautiful))
print("Cosine Similarity: ", cosine)

print("Model Similarity:  ", modelword2vec.wv.similarity('baby','beautiful'))

Cosine Similarity:  0.99867696
Model Similarity:   0.99867696


In [17]:
modelword2vec.wv.most_similar('baby', topn=20)

[('know', 0.999595046043396),
 ('how', 0.9995772838592529),
 ('more', 0.9995602369308472),
 ('ever', 0.9995595812797546),
 ('miss', 0.9995383024215698),
 ('he', 0.9995322823524475),
 ('really', 0.9995176196098328),
 ('keep', 0.9995153546333313),
 ('ai', 0.999511182308197),
 ('myself', 0.9995027780532837),
 ('another', 0.9995015859603882),
 ('even', 0.999493420124054),
 ('did', 0.9994879961013794),
 ('say', 0.9994861483573914),
 ('is', 0.9994827508926392),
 ('why', 0.9994769096374512),
 ('so', 0.9994679093360901),
 ('leave', 0.9994663000106812),
 ('when', 0.9994619488716125),
 ('would', 0.9994558095932007)]

# เรียกใช้ pre-trained model ของ word2vec 

In [18]:
import gensim.downloader as api
#โหลด pre-trained Word2Vec จาก Google News
model = api.load('word2vec-google-news-300')

# Synonym

In [27]:
print(f"1.คำศัพท์ Synonym ที่แปลว่า สวย งดงาม ดีงาม เลิศ : {model.similarity('Charming', 'Pretty')}")
print(f"2.คำศัพท์ Synonym ที่แปลว่า มีความสุข ปลื้ม สบายใจ สุขสม : {model.similarity('Happy', 'Energetic')}")
print(f"3.คำศัพท์ synonym ที่แปลว่า ความโกรธ ไม่ชอบ ขัดเคือง เสียใจ : {model.similarity('Anger', 'Aggravated')}")
print(f"4.คำศัพท์ synonym ที่แปลว่า กลัว, เกรง, เกลียด, ตระหนก,เครียด : {model.similarity('Fear', 'Anxious')}")
print(f"5.คำศัพท์ Synonym ที่แปลว่า ความสับสน ยุ่งเหยิง งงงวย ความทุลักทุเล : {model.similarity('Confusion', 'Ambivalent')}")
print(f"6.คำศัพท์ Synonym ที่แปลว่า ภาวะซึมเศร้า ห่อเหี่ยว ท้อถอย : {model.similarity('Depression', 'Dejected')}")
print(f"7.คำศัพท์ Synonym ที่แปลว่า ตรงกันข้าม แปลกประหลาด : {model.similarity('Contradictory', 'Capricious')}")
print(f"8.คำศัพท์ Synonym ที่แปลว่า ความสามารถ : {model.similarity('abilities', 'capability')}")
print(f"9.คำศัพท์ Synonym ที่แปลว่า ได้รับ: {model.similarity('get', 'accept')}")
print(f"10.คำศัพท์ Synonym ที่แปลว่า เห็นด้วย : {model.similarity('agree', 'assent')}")

1.คำศัพท์ Synonym ที่แปลว่า สวย งดงาม ดีงาม เลิศ : 0.26816192269325256
2.คำศัพท์ Synonym ที่แปลว่า มีความสุข ปลื้ม สบายใจ สุขสม : 0.23426812887191772
3.คำศัพท์ synonym ที่แปลว่า ความโกรธ ไม่ชอบ ขัดเคือง เสียใจ : 0.22215387225151062
4.คำศัพท์ synonym ที่แปลว่า กลัว, เกรง, เกลียด, ตระหนก,เครียด : 0.30024874210357666
5.คำศัพท์ Synonym ที่แปลว่า ความสับสน ยุ่งเหยิง งงงวย ความทุลักทุเล : 0.31179922819137573
6.คำศัพท์ Synonym ที่แปลว่า ภาวะซึมเศร้า ห่อเหี่ยว ท้อถอย : 0.01573185995221138
7.คำศัพท์ Synonym ที่แปลว่า ตรงกันข้าม แปลกประหลาด : 0.23361623287200928
8.คำศัพท์ Synonym ที่แปลว่า ความสามารถ : 0.46027469635009766
9.คำศัพท์ Synonym ที่แปลว่า ได้รับ: 0.3165584206581116
10.คำศัพท์ Synonym ที่แปลว่า เห็นด้วย : 0.1876089870929718


In [61]:
# ข้อมูลทดสอบ
test_data = [
    {'word1': 'Fear', 'word2': 'Anxious', 'expected_similarity': 0.3},
    {'word1': 'Charming', 'word2': 'Pretty', 'expected_similarity': 0.3},
    {'word1': 'Happy', 'word2': 'Energetic', 'expected_similarity': 0.3},
    {'word1': 'Anger', 'word2': 'Aggravated', 'expected_similarity': 0.3},
    {'word1': 'Confusion', 'word2': 'Ambivalent', 'expected_similarity': 0.3},
    {'word1': 'Depression', 'word2': 'Dejected', 'expected_similarity': 0.3},
    {'word1': 'Contradictory', 'word2': 'Capricious', 'expected_similarity': 0.3},
    {'word1': 'abilities', 'word2': 'capability', 'expected_similarity': 0.3},
    {'word1': 'get', 'word2': 'accept', 'expected_similarity': 0.3},
    {'word1': 'agree', 'word2': 'assent', 'expected_similarity': 0.3} 
]


# ทดสอบโมเดล
correct_predictions = 0
total_tests = len(test_data)

for test_case in test_data:
    word1 = test_case['word1']
    word2 = test_case['word2']
    expected_similarity = test_case['expected_similarity']

    try:
        # ทำนายความคล้ายคลึง
        similarity_score = model.similarity(word1, word2)

        # เปรียบเทียบกับค่าความคล้ายที่คาดหวัง
        if similarity_score >= expected_similarity:
            correct_predictions += 1

    except KeyError as e:
        print(f"Word '{e}' not present in the vocabulary.")

# คำนวณค่า accuracy
accuracy = correct_predictions / total_tests
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 40.00%


แหล่งที่มาโจทย์
- https://xchangeenglish.com/blog/find-vocabulary-synonym/
- https://interpass.in.th/common-synonyms-found-in-exam/

# Analogy

In [55]:
analogy_result1 = model.most_similar(positive=['court', 'general'], negative=['judge'], topn=1)
print(f"1.general : {analogy_result1[0][0]}")

analogy_result2 = model.most_similar(positive=['idle', 'impartial'], negative=['industrious'], topn=1)
print(f"2.impartial : {analogy_result2[0][0]}")

analogy_result3 = model.most_similar(positive=['hand', 'mask'], negative=['gauntlet'], topn=1)
print(f"3.mask : {analogy_result3[0][0]}")

analogy_result4 = model.most_similar(positive=['bread', 'splinter'], negative=['crumb'], topn=1)
print(f"4.splinter : {analogy_result4[0][0]}")

analogy_result5 = model.most_similar(positive=['emotion', 'pattern'], negative=['depression'], topn=1)
print(f"5.pattern : {analogy_result5[0][0]}")

analogy_result6 = model.most_similar(positive=['bad', 'brave'], negative=['good'], topn=1)
print(f"6.brave : {analogy_result6[0][0]}")

analogy_result7 = model.most_similar(positive=['build', 'author'], negative=['carpenter'], topn=1)
print(f"7.author : {analogy_result7[0][0]}")

analogy_result8 = model.most_similar(positive=['small', 'huge'], negative=['short'], topn=1)
print(f"8.huge : {analogy_result8[0][0]}")

analogy_result9 = model.most_similar(positive=['eye', 'computer'], negative=['face'], topn=1)
print(f"9.computer : {analogy_result9[0][0]}")

analogy_result10 = model.most_similar(positive=['accident', 'anger'], negative=['careless'], topn=1)
print(f"10.anger : {analogy_result10[0][0]}")

analogy_result11 = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(f"11.king : {analogy_result11[0][0]}")

1.general : gen_eral
2.impartial : unbiased
3.mask : forehead
4.splinter : splinter_faction
5.pattern : patterns
6.brave : courageous
7.author : develop
8.huge : large
9.computer : computers
10.anger : crash
11.king : queen


In [66]:
from gensim.models import KeyedVectors

#โจทย์ Analogy 10 ข้อ
test_pairs = [
    ("judge", "court", "general", "gen_eral"),
    ("industrious", "idle", "impartial", "unbiased"),
    ("gauntlet", "hand", "mask", "forehead"),
    ("crumb", "bread", "splinter", "splinter_faction"),
    ("depression", "emotion", "pattern", "patterns"),
    ("good", "bad", "brave", "courageous"),
    ("carpenter", "build", "author", "develop"),
    ("short", "small", "huge", "large"),
    ("face", "eye", "computer", "computers"),
    ("careless", "accident", "anger", "crash")
    
]

# นับจำนวนคำทดสอบที่ทำนายถูกต้อง
correct_predictions = 0

# ทดสอบโมเดล
for pair in test_pairs:
    word1, word1_similar, word2, word2_similar = pair
    try:
        # ทำนายความคล้ายคลึง
        similarity_score = model.similarity(word2, word2_similar)
        
        # เปรียบเทียบกับคำตอบที่คาดหวัง
        if similarity_score > 0.5:  
            correct_predictions += 1
    except KeyError as e:
        print(f"Word '{e}' not present in the vocabulary.")

# คำนวณค่า accuracy
accuracy = correct_predictions / len(test_pairs)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 70.00%


แหล่งที่มาโจทย์
- https://www.opendurian.com/exercises/gat_eng_mar_52/21/
- https://wordwall.net/th/resource/5056295/analogy

# Doesn't match 

In [35]:
print(f"1.The word that doesn't match: {model.doesnt_match(['hoard', 'collect', 'eradicate', 'accumulate'])}")
print(f"2.The word that doesn't match: {model.doesnt_match(['offspring', 'beginning', 'commencement', 'infancy'])}")
print(f"3.The word that doesn't match: {model.doesnt_match(['advertise', 'entangle', 'publicize', 'circulate'])}")
print(f"4.The word that doesn't match: {model.doesnt_match(['attraction', 'enticement', 'priority', 'lure'])}")
print(f"5.The word that doesn't match: {model.doesnt_match(['contingent', 'coherent', 'accidental', 'haphazard'])}")
print(f"6.The word that doesn't match: {model.doesnt_match(['disentangle', 'solve', 'resolve', 'ponder'])}")
print(f"7.The word that doesn't match: {model.doesnt_match(['composed', 'arbitrary', 'random', 'indulgent'])}")
print(f"8.The word that doesn't match: {model.doesnt_match(['prosperity', 'accommodation', 'home', 'residence'])}")
print(f"9.The word that doesn't match: {model.doesnt_match(['ear', 'lip', 'nose', 'chest'])}")
print(f"10.The word that doesn't match: {model.doesnt_match(['copper', 'iron', 'aluminum', 'carbon'])}")

1.The word that doesn't match: eradicate
2.The word that doesn't match: offspring
3.The word that doesn't match: entangle
4.The word that doesn't match: priority
5.The word that doesn't match: contingent
6.The word that doesn't match: ponder
7.The word that doesn't match: composed
8.The word that doesn't match: prosperity
9.The word that doesn't match: chest
10.The word that doesn't match: carbon


In [67]:
# Sample dataset with labeled examples
doesnt_match_data = [
    (['hoard', 'collect', 'eradicate', 'accumulate'], 'eradicate'),
    (['offspring', 'beginning', 'commencement', 'infancy'], 'offspring'),
    (['advertise', 'entangle', 'publicize', 'circulate'], 'entangle'),
    (['attraction', 'enticement', 'priority', 'lure'], 'priority'),
    (['contingent', 'coherent', 'accidental', 'haphazard'], 'coherent'),
    (['disentangle', 'solve', 'resolve', 'ponder'], 'ponder'),
    (['composed', 'arbitrary', 'random', 'indulgent'], 'composed'),
    (['prosperity', 'accommodation', 'home', 'residence'], 'prosperity'),
    (['ear', 'lip', 'nose', 'chest'], 'chest'),
    (['copper', 'iron', 'aluminum', 'carbon'], 'carbon')
]

# Test the "Doesn't match" task and calculate accuracy
correct_predictions = 0
total_examples = len(doesnt_match_data)

for example, expected_answer in doesnt_match_data:
    try:
        result = model.doesnt_match(example)
        
        # Check if the result matches the expected answer
        if result == expected_answer:
            correct_predictions += 1
    except ValueError as e:
        print(f"Error: {e}")

# Calculate accuracy
accuracy = correct_predictions / total_examples
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 90.00%


แหล่งที่มาโจทย์
- https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fpitukpong.files.wordpress.com%2F2013%2F07%2Fe0b882e0b989e0b8ade0b8aae0b8ade0b89a-analogy-e0b980e0b89be0b987e0b899e0b882e0b989e0b8ade0b8aae0b8ade0b89ae0b897e0b8b5e0b988e0b8a7e0b8b1.doc&wdOrigin=BROWSELINK